In [1]:
import requests
import os
from decimal import Decimal

In [2]:
api_token = os.environ.get("TILTIFY_TOKEN")
campaign_id = 155357

In [3]:
rewards_dict = {}

url = f"/api/v3/campaigns/{campaign_id}/rewards?count=100"

response = requests.get(
    "https://tiltify.com" + url,
    headers={"Authorization": "Bearer {}".format(api_token)},
)
response = response.json()

for row in response["data"]:
    rewards_dict[row["id"]] = row

In [4]:
rewards_dict[132555]

{'id': 132555,
 'type': 'Reward',
 'name': 'P4A Digital Wallpapers!',
 'campaignId': 155357,
 'amount': 6.0,
 'kind': 'virtual',
 'quantity': None,
 'remaining': None,
 'fairMarketValue': None,
 'description': 'Enjoy some festive P4A wallpapers for your computer AND phone! Desktop wallpaper designed by nerdfighter Alys and phone wallpapers by Daniel Jennings. Included in the digital download bundle.',
 'currency': 'USD',
 'shippingAddressRequired': False,
 'shippingNote': None,
 'image': {'src': 'https://assets.tiltify.com/uploads/reward/image/132555/blob-70c69b28-aa8f-4658-a54e-a6e4426fc151.png',
  'alt': 'Enjoy some festive P4A wallpapers for your computer AND phone! Desktop wallpaper designed by nerdfighter Alys and phone wallpapers by Daniel Jennings. Included in the digital download bundle.',
  'width': 600,
  'height': 600},
 'active': True,
 'startsAt': 0,
 'createdAt': 1645219259000,
 'updatedAt': 1645653143000,
 'retiredAt': None,
 'activatedAt': 1645219263000,
 'deactivatedAt

In [5]:
donations_dict = {}

url = f"/api/v3/campaigns/{campaign_id}/donations?count=100"

while url:
    print(url)
    response = requests.get(
        "https://tiltify.com" + url,
        headers={"Authorization": "Bearer {}".format(api_token)},
    )

    response = response.json()

    for row in response["data"]:
        donations_dict[row["id"]] = row

    url = response["links"]["prev"]

/api/v3/campaigns/155357/donations?count=100
/api/v3/campaigns/155357/donations?count=100&before=5633493
/api/v3/campaigns/155357/donations?count=100&before=5633310
/api/v3/campaigns/155357/donations?count=100&before=5633127
/api/v3/campaigns/155357/donations?count=100&before=5632954
/api/v3/campaigns/155357/donations?count=100&before=5632776
/api/v3/campaigns/155357/donations?count=100&before=5632507
/api/v3/campaigns/155357/donations?count=100&before=5632343
/api/v3/campaigns/155357/donations?count=100&before=5632305


In [6]:
donations = pd.DataFrame.from_records(list(donations_dict.values()))
donations["rewardId"] = donations["rewardId"].astype("Int64")
donations["rewardName"] = (
    donations["rewardId"]
    .map(lambda x: None if pd.isna(x) else rewards_dict[x]["name"])
    .astype("string")
)
donations["rewardPrice"] = (
    donations["rewardId"]
    .map(lambda x: None if pd.isna(x) else rewards_dict[x]["amount"])
    .astype(float)
)
donations

,id,amount,name,comment,completedAt,updatedAt,rewardId,sustained,rewardName,rewardPrice
0,5633757,65.0,mpalka,"The FIRST EVER P4A Hoodie! Thanks, Maia! :D",1645653792000,1645653793000,131613,False,P4A hoodie,65.0
1,5633755,60.0,Amy,None,1645653683000,1645653683000,132546,False,Digital Download Bundle,60.0
2,5633756,40.0,becboops,None,1645653666000,1645653666000,132914,False,P4A Dice,40.0
3,5633754,60.0,Clint Adams,None,1645653614000,1645653615000,132546,False,Digital Download Bundle,60.0
4,5633753,40.0,KennaB,Yay John Green and brother!,1645653590000,1645653591000,131611,False,P4A 2022 Sticker,10.0
...,...,...,...,...,...,...,...,...,...,...
721,5632313,5.0,Fallen,Just wanted to say hi,1645592756000,1645592757000,<NA>,False,<NA>,NaN
722,5632312,10.0,Em Gunter,"In honor of Paul Farmer, a man who inspired ma...",1645592756000,1645592756000,131611,False,P4A 2022 Sticker,10.0
723,5632307,10.0,Jenn Marker,Let's go 2022!!!,1645592576000,1645592576000,131611,False,P4A 2022 Sticker,10.0
724,5632306,1.0,Anonymous,P4A Test!,1645592543000,1645592543000,<NA>,False,<NA>,NaN


In [7]:
print("Total donations: ", len(donations))

Total donations:  726


In [8]:
donations["is_anonymous"] = donations["name"] == "Anonymous"
anonymous = donations.groupby("is_anonymous").aggregate(
    count=("is_anonymous", "count"), sum=("amount", "sum")
)
anonymous.reset_index(inplace=True)
anonymous.rename(
    columns={"sum": "Raised", "count": "Count", "is_anonymous": "Who"}, inplace=True
)
anonymous["Who"] = anonymous["Who"].map(lambda x: "Anonymous" if x else "Other")
anonymous.sort_values("Who", inplace=True)
anonymous

,Who,Count,Raised
1,Anonymous,135,5720.00
0,Other,591,25782.69


In [9]:
rewards_stats = donations.groupby("rewardId", dropna=False).aggregate(
    count=("id", "count"), raised=("amount", "sum")
)
rewards_stats["name"] = rewards_stats.index.map(
    lambda x: None if pd.isna(x) else rewards_dict[x]["name"]
).astype("string")
rewards_stats["basePrice"] = rewards_stats.index.map(
    lambda x: None if pd.isna(x) else rewards_dict[x]["amount"]
).astype(float)
rewards_stats.reset_index(inplace=True, drop=False)
rewards_stats["raisedOverPrice"] = rewards_stats["raised"] - (
    rewards_stats["basePrice"] * rewards_stats["count"]
)
rewards_stats["raisedOfTotal"] = round(
    rewards_stats["raised"] / rewards_stats["raised"].sum() * 100, 2
)
rewards_stats.sort_values(by="raisedOfTotal", inplace=True, ascending=False)
rewards_stats.reset_index(drop=True, inplace=True)
rewards_stats.rename(
    columns={
        "rewardId": "id",
    },
    inplace=True,
)
rewards_stats = rewards_stats[
    ["id", "name", "basePrice", "count", "raisedOverPrice", "raisedOfTotal"]
]
rewards_stats["id"] = rewards_stats["id"].astype("string").fillna("")
rewards_stats["name"] = rewards_stats["name"].fillna("<No Reward>")
rewards_stats

,id,name,basePrice,count,raisedOverPrice,raisedOfTotal
0,132546,Digital Download Bundle,60.0,191,357.68,37.51
1,131613,P4A hoodie,65.0,46,140.00,9.94
2,133118,P4A Puzzle!,40.0,47,30.00,6.06
3,132914,P4A Dice,40.0,41,0.00,5.21
4,131609,P4A 2022 Commemorative Coin,24.0,49,108.00,4.08
5,132575,John's Tiktok Drafts!,15.0,63,330.00,4.05
6,132875,P4A 2022 Socks,50.0,22,0.00,3.49
7,133120,P4A Playing Cards,30.0,34,20.00,3.30
8,130532,P4A Quilt,900.0,1,0.00,2.86
9,,<No Reward>,NaN,26,NaN,2.65


In [10]:
donations["decimals"] = (
    donations["amount"]
    .map(Decimal)
    .map(lambda x: x.quantize(Decimal(".01")))
    .map(lambda x: x % 1)
    .astype("string")
    .str.split(".", expand=True)[1]
)

In [11]:
donations.groupby("decimals").agg(count=("id", "count")).reset_index().sort_values(
    "count", ascending=False
)

,decimals,count
0,00,723
1,32,1
2,68,1
3,69,1
